In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm
import datasets
from typing import List, Tuple
from datasets import load_dataset

class PromptDataset(Dataset):
    def __init__(self, texts: List[str], labels: List[int], tokenizer, max_length=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]

        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

def prepare_data(gsm8k: datasets.Dataset, humaneval: datasets.Dataset) -> Tuple[List[str], List[int]]:


    gsm_prompts = [str(item['question']) for item in gsm8k]
    humaneval_prompts = [str(item['prompt']) for item in humaneval]
    
    gsm_labels = [0] * len(gsm_prompts)
    humaneval_labels = [1] * len(humaneval_prompts)
    
    all_prompts = gsm_prompts + humaneval_prompts
    all_labels = gsm_labels + humaneval_labels
    
    return all_prompts, all_labels

def train_bert_classifier(prompts: List[str], labels: List[int], 
                         batch_size=16, num_epochs=3, learning_rate=2e-5):

    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
    
    train_texts, val_texts, train_labels, val_labels = train_test_split(
        prompts, labels, test_size=0.2, random_state=42
    )
    
    train_dataset = PromptDataset(train_texts, train_labels, tokenizer)
    val_dataset = PromptDataset(val_texts, val_labels, tokenizer)
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    
    for epoch in range(num_epochs):
        print(f'\nEpoch {epoch + 1}/{num_epochs}')
        
        model.train()
        train_loss = 0
        train_steps = 0
        
        for batch in tqdm(train_loader, desc='Training'):
            optimizer.zero_grad()
            
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            train_steps += 1
        
        avg_train_loss = train_loss / train_steps
        print(f'Average training loss: {avg_train_loss:.4f}')
        
        model.eval()
        val_loss = 0
        val_steps = 0
        correct = 0
        total = 0
        
        with torch.no_grad():
            for batch in tqdm(val_loader, desc='Validation'):
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['label'].to(device)
                
                outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss
                
                val_loss += loss.item()
                val_steps += 1
                
                predictions = torch.argmax(outputs.logits, dim=1)
                correct += (predictions == labels).sum().item()
                total += labels.size(0)
        
        avg_val_loss = val_loss / val_steps
        accuracy = correct / total
        print(f'Average validation loss: {avg_val_loss:.4f}')
        print(f'Validation accuracy: {accuracy:.4f}')
    
    return model, tokenizer

if __name__ == "__main__":
    gsm8k = load_dataset("gsm8k", "main", split="train")
    humaneval = load_dataset("openai_humaneval", split="test")
    
    prompts, labels = prepare_data(gsm8k, humaneval)
    
    model, tokenizer = train_bert_classifier(prompts, labels)
    
    model.save_pretrained("../models/prompt_classifier")
    tokenizer.save_pretrained("../models/prompt_classifier")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/dmanu006/.cache/pypoetry/virtualenvs/moelm-djswBcic-py3.10/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(



Epoch 1/3


Training: 100%|██████████| 382/382 [01:45<00:00,  3.60it/s]


Average training loss: 0.0235


Validation: 100%|██████████| 96/96 [00:12<00:00,  7.61it/s]


...

Average validation loss: 0.0001
Validation accuracy: 1.0000

Epoch 3/3


Training: 100%|██████████| 382/382 [02:54<00:00,  2.19it/s]


Average training loss: 0.0001


Validation: 100%|██████████| 96/96 [00:17<00:00,  5.51it/s]


Average validation loss: 0.0001
Validation accuracy: 1.0000


In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

def predict_dataset(text: str, model_path: str = "prompt_classifier") -> dict:

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = BertForSequenceClassification.from_pretrained(model_path)
    tokenizer = BertTokenizer.from_pretrained(model_path)
    model.to(device)
    model.eval()
    
    encoding = tokenizer(
        text,
        add_special_tokens=True,
        max_length=512,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        probabilities = torch.softmax(outputs.logits, dim=1)
        prediction = torch.argmax(probabilities, dim=1).item()
        confidence = probabilities[0][prediction].item()
    
    dataset_mapping = {0: "GSM8K (Math)", 1: "HumanEval (Programming)"}
    result = {
        "predicted_dataset": dataset_mapping[prediction],
        "confidence": f"{confidence:.2%}",
        "probabilities": {
            "GSM8K": f"{probabilities[0][0].item():.2%}",
            "HumanEval": f"{probabilities[0][1].item():.2%}"
        }
    }
    
    return result

if __name__ == "__main__":
    math_prompt = """
    Janet has 3 apples. She buys 2 more apples from the store. 
    How many apples does Janet have in total?
    """
    
    programming_prompt = """
    def add_numbers(a: int, b: int) -> int:
        \"\"\"
        Add two integers and return their sum.
        
        Args:
            a: first integer
            b: second integer
            
        Returns:
            The sum of a and b
        \"\"\"
    """
    
    math_result = predict_dataset(math_prompt)
    programming_result = predict_dataset(programming_prompt)
    
    print("\nMath Problem Prediction:")
    print(f"Predicted Dataset: {math_result['predicted_dataset']}")
    print(f"Confidence: {math_result['confidence']}")
    print("Probabilities:", math_result['probabilities'])
    
    print("\nProgramming Problem Prediction:")
    print(f"Predicted Dataset: {programming_result['predicted_dataset']}")
    print(f"Confidence: {programming_result['confidence']}")
    print("Probabilities:", programming_result['probabilities'])


Math Problem Prediction:
Predicted Dataset: GSM8K (Math)
Confidence: 99.99%
Probabilities: {'GSM8K': '99.99%', 'HumanEval': '0.01%'}

Programming Problem Prediction:
Predicted Dataset: HumanEval (Programming)
Confidence: 99.97%
Probabilities: {'GSM8K': '0.03%', 'HumanEval': '99.97%'}
